## Supporting workings - Navigating census data

As support for the main workbook, I first need an effective way of selecting which categories to select in the first instance from the extremely large datasets. From the raw metadata file for the 2016 datapack, there was 15536 different data points for each statistical area across all 59 tables. Some of these were totals of other lines, some were the total population, some had combinations of classifications available elsewhere in the overall population, etc. so a mechanism of refinement is clearly necessary.

Ideally I would like to be able to import data based on some simple aggregations where possible (e.g. by sex), or otherwise taking the "highest" level data possible given the input constraints. E.g. I would like to say "Import data from tables G09, G52 and G53, but split by sex where possible", if "sex" is not available in one of the tables, it will still return the data from the table, but from the most aggregated column(s) only. Additionally, if "Sex" is only available as a sub-category of another split (e.g. the measure is "Sex by Age") I would like to be able to perform my own aggregation, to abstract away the unneccessary other details imbedded in the data (e.g. sum all the "Age" categories together, grouping by sex).

In [1]:
# Import statements
# Declare Imports
import numpy as np
import pandas as pd
import os
import re
import operator
import csv

# Set a variable for current notebook's path for various loading/saving mechanisms
nb_path = os.getcwd()

In [2]:
# Generalise this importation method to allow easy imports based on folder name (for SA level) 
# and a list of datapack files you want to amalgamate into a single dataframe
def load_census_csv(table_list, statistical_area_code):
    statistical_area_code = statistical_area_code.upper()
    for index, table in enumerate(table_list):
    
        if index==0:
            df = pd.read_csv('{}\Data\{}\AUST\\2016Census_{}_AUS_{}.csv'.format(nb_path,
                                                                                statistical_area_code,
                                                                                table,
                                                                                statistical_area_code
                                                                               ),
                                       engine='python')
        else:
            temp_df = pd.read_csv('{}\Data\{}\AUST\\2016Census_{}_AUS_{}.csv'.format(nb_path,
                                                                                statistical_area_code,
                                                                                table,
                                                                                statistical_area_code
                                                                               ),
                                       engine='python')
            merge_col = df.columns[0]
            df = pd.merge(df, temp_df, on=merge_col)
    
    return df

### Begin importing and exploring data

In [2]:
# Import metadata sheets
df_meta_tables = pd.read_excel('{}\Data\Metadata\Metadata_2016_GCP_DataPack.xlsx'.format(nb_path),
                               sheet_name = 'Table number, name, population',
                               skiprows=9)
df_meta_measures = pd.read_excel('{}\Data\Metadata\Metadata_2016_GCP_DataPack.xlsx'.format(nb_path),
                               sheet_name = 'Cell descriptors information',
                               skiprows=10)

In [3]:
df_meta_tables.tail()

,Table number,Table name,Table population
54,G55,Total Family Income (Weekly) by Labour Force S...,Couple families with children
55,G56,Total Family Income (Weekly) by Labour Force S...,One parent families
56,G57,Occupation by Age by Sex,Employed persons aged 15 years and over
57,G58,Occupation by Hours Worked by Sex,Employed persons aged 15 years and over
58,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over


In [4]:
df_meta_measures['Table number'] = df_meta_measures['DataPack file'].str[:3]
df_meta_measures = pd.merge(df_meta_measures, df_meta_tables, on='Table number')
df_meta_measures.shape

(15535, 9)

In [5]:
df_meta_measures.tail(20)

,Sequential,Short,Long,DataPack file,Profile table,Column heading description in profile,Table number,Table name,Table population
15515,G15516,Three_met_Bs_2_ot_met_ex_tr_F,Three_methods_Bus_and_two_other_methods_exclud...,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15516,G15517,Three_met_Bs_2_ot_met_ex_tr_P,Three_methods_Bus_and_two_other_methods_exclud...,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15517,G15518,Three_meth_Othr_three_meth_M,Three_methods_Other_three_methods_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15518,G15519,Three_meth_Othr_three_meth_F,Three_methods_Other_three_methods_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15519,G15520,Three_meth_Othr_three_meth_P,Three_methods_Other_three_methods_Persons,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15520,G15521,Three_meth_Tot_three_meth_M,Three_methods_Total_three_methods_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15521,G15522,Three_meth_Tot_three_meth_F,Three_methods_Total_three_methods_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15522,G15523,Three_meth_Tot_three_meth_P,Three_methods_Total_three_methods_Persons,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15523,G15524,Worked_home_M,Worked_at_home_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15524,G15525,Worked_home_F,Worked_at_home_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over


#### Quick notes:
I noticed that very conveniently a lot of the capitalisation in the "Long" field works as a delimiter between different population characteristics. Working through some exceptions below, split the Long fields based on capitalisation (and a couple of other hueristics) to get down to a much shorter list of 911 characteristics. I turned this information into a csv file which I then did some tedious review on in order to come up with a much shorter list of 61 categories and created a new "Meta" file which I import after this section.

In [6]:
def lower_all_except_first_char(str_item):
    str_replace = '{}{}'.format(str_item[:1],str_item[1:].lower())
    df_meta_measures['Long'] = df_meta_measures['Long'].str.replace(str_item, str_replace)

In [7]:
replace_capitalisation_list = ['Not_stated',
                               'None',
                               'No_children',
                               'Aboriginal_and_or_Torres_Strait_Islander',
                               'Both_Aboriginal_and_Torres_Strait_Islander',
                               'Torres_Strait_Islander',
                               'Non_Indigenous',
                               'New_Zealand',
                               'South_Africa',
                               'Sri_Lanka',
                               'Bosnia_and_Herzegovina',
                               'China_excludes_SARs_and_Taiwan',
                               'Hong_Kong_SAR_of_China',
                               'Korea_Republic_of_South',
                               'Northern_Ireland',
                               'Papua_New_Guinea',
                               'South_Eastern_Europe',
                               'Total_Responses',
                               'Census_Night',
                               'Elsewhere_in_Australia',
                               'Birthplace_Australia',
                               'Birthplace_Elsewhere',
                               'Other_Language',
                               'Age_of_Persons',
                               'Count_of_Persons',
                               'Average_number_of_Persons_per_bedroom',
                               'Visitor_from_Different_SA2',
                               'Visitor_from_Same_Statistical_Area_Level_2_SA2',
                               'Western_Australia',
                               'Northern_Territory',
                               'Australian_Capital_Territory',
                               'South_Australia',
                               'New_South_Wales',
                               'Other_Territories',
                               'China_excl_SARs_and_Taiwan',
                               'United_Kingdom_Channel_Islands_and_Isle_of_Man',
                               'The_Former_Yugoslav_Republic_of_Macedonia',
                               'United_States_of_America',
                               'Year_of_arrival_Before',
                               'Speaks_English',
                               'speaks_English',
                               'Proficiency_in_English',
                               'Proficiency_in_english',
                               '_Before_2000',
                               'Total_Year_of_arrival_not_stated',
                               'Australian_Indigenous_Languages',
                               'Chinese_Languages_Cantonese',
                               'Chinese_Languages_Mandarin',
                               'Chinese_Languages_Other',
                               'Chinese_languages_Other',
                               'Chinese_Languages_Total',
                               'Indo_Aryan_Languages_Bengali',
                               'Indo_Aryan_Languages_Hindi',
                               'Indo_Aryan_Languages_Punjabi',
                               'Indo_Aryan_Languages_Sinhalese',
                               'Indo_Aryan_Languages_Urdu',
                               'Indo_Aryan_Languages_Other',
                               'Indo_Aryan_Languages_Total',
                               'Persian_excluding_Dari',
                               'Southeast_Asian_Austronesian_Languages_Filipino',
                               'Southeast_Asian_Austronesian_Languages_Indonesian',
                               'Southeast_Asian_Austronesian_Languages_Tagalog',
                               'Southeast_Asian_Austronesian_Languages_Other',
                               'Southeast_Asian_Austronesian_Languages_Total',
                               'Christianity_Anglican',
                               'Christianity_Assyrian_Apostolic',
                               'Christianity_Baptist',
                               'Christianity_Brethren',
                               'Christianity_Catholic',
                               'Christianity_Churches_of_Christ',
                               'Christianity_Eastern_Orthodox',
                               'Christianity_Jehovahs_Witnesses',
                               'Christianity_Latter_day_Saints',
                               'Christianity_Lutheran',
                               'Christianity_Oriental_Orthodox',
                               'Christianity_Other_Protestant',
                               'Christianity_Pentecostal',
                               'Christianity_Presbyterian_and_Reformed',
                               'Christianity_Salvation_Army',
                               'Christianity_Seventh_day_Adventist',
                               'Christianity_Uniting_Church',
                               'Christianity_Christianity_nfd',
                               'Christianity_Other_Christian',
                               'Christianity_Total',
                               'Other_Religions_Australian_Aboriginal_Traditional_Religions',
                               'Other_Religions_Sikhism',
                               'Other_Religions_Other',
                               'Other_Religions_Total',
                               'Secular_Beliefs_and_Other_Spiritual_Beliefs_and_No_Religious_Affiliation_No_Religion_So_Described',
                               'Secular_Beliefs_and_Other_Spiritual_Beliefs_and_No_Religious_Affiliation_Secular_Beliefs',
                               'Secular_Beliefs_and_Other_Spiritual_Beliefs_and_No_Religious_Affiliation_Other_Spiritual_Beliefs',
                               'Secular_Beliefs_and_Other_Spiritual_Beliefs_and_No_Religious_Affiliation_Total',
                               'Infants_Primary',
                               'Other_Non_Government',
                               'Technical_or_Further_Educational_institution',
                               'Full_Part_time',
                               'University_or_other_Tertiary_Institution',
                               'Males_Negative_Nil_income', #maybe look at this whole section
                               'Females_Negative_Nil_income', #maybe look at this whole section
                               'Persons_Negative_Nil_income', #maybe look at this whole section
                               'Males_Personal_income_not_stated',#maybe look at this whole section
                               'Feales_Personal_income_not_stated',#maybe look at this whole section
                               'Persons_Personal_income_not_stated',#maybe look at this whole section
                               'Cared_for_Own',
                               'Cared_for_Other',
                               'Visitor_from_within_Australia',
                               'born_in_Australia',
                               'Real_Estate_Agent',
                               'with_Children',
                               'with__No_children',
                               'Flat_or_Apartment',
                               'Graduate_Diploma_and_Graduate_Certificate_Level_Graduate_Diploma_Level', 
                               'Advanced_Diploma_and_Diploma_Level_Advanced_Diploma_and_Associate_Degree_Level',
                               'Advanced_Diploma_and_Diploma_Advanced_Diploma_and_Associate_Degree_Level',
                               'Certificate_Level_Certificate_III_and_IV_Level',
                               'Certificate_Level_Certificate_I_and_II_Level',
                               'Certificate_Level_Certificate_Level_nfd',
                               'Graduate_Diploma_and_Graduate_Certificate_Level',
                               'Advanced_Diploma_and_Diploma_Level_Diploma_Level',
                               'Occupation_Inadequately_described',
                               'Postgraduate_Degree_Level',
                               'Master_Degree_Level',
                               'Doctoral_Degree_Level',
                               'Certificate_I_and_II_Level',
                               'Certificate_III_and_IV_Level',
                               'Advanced_Diploma_and_Diploma_Level',
                               'Graduate_Certificate_Level',
                               'Bachelor_Degree_Level',
                               'Certificate_Level',
                               'Different_SA2',
                               'Same_Statistical_Area_Level_2',
                               'Lone_Parent',
                               'Worked_Full_Time',
                               'Worked_Part_Time',
                               'Away_From_Work',
                               'Age_Of_Dependent_Children',
                               '4_Years',
                               '9_Years',
                               '2_Years',
                               '7_Years',
                               '0_Years',
                               'Hours_Worked',
                               'Labour_Force_Status_Not_Stated',
                               'Not_In_The_Labour_Force',
                               'Labour_Force',
                               'Looking_For_Full_Time_Work',
                               'Looking_For_Part_Time_Work',
                               'Natural_and_Physical_Sciences',
                               'Information_Technology',
                               'Engineering_and_Related_Technologies',
                               'Architecture_and_Building',
                               'Agriculture_Environmental_and_Related_Studies',
                               'Management_and_Commerce',
                               'Society_and_Culture',
                               'Creative_Arts',
                               'Food_Hospitality_and_Personal_Services',
                               'Mixed_Field_Programmes',
                               'Male_Parent',
                               'Looking_For',
                               'Hours_Worked_Not_Stated',
                               'Inadequately_described_Not_stated',
                               'Number_of_hours_worked_None',
                               'Number_of_hours_worked_Not_stated',
                               'Dependent_children_In_Couple_Families',
                               'Negative_Nil',
                               'Never_Married',
                               '_Census_night',
                               'Speaks_English',
                               'Speaks_english',
                               'Speaks_other'
                              ]

# note: there were still some areas I missed and had to clean up in the tedious manual work on the csv file

In [8]:
replace_capitalisation_list.sort(key = len, reverse=True)
for correction in replace_capitalisation_list:
    lower_all_except_first_char(correction)

In [9]:
df_meta_measures.tail(20)

,Sequential,Short,Long,DataPack file,Profile table,Column heading description in profile,Table number,Table name,Table population
15515,G15516,Three_met_Bs_2_ot_met_ex_tr_F,Three_methods_Bus_and_two_other_methods_exclud...,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15516,G15517,Three_met_Bs_2_ot_met_ex_tr_P,Three_methods_Bus_and_two_other_methods_exclud...,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15517,G15518,Three_meth_Othr_three_meth_M,Three_methods_Other_three_methods_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15518,G15519,Three_meth_Othr_three_meth_F,Three_methods_Other_three_methods_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15519,G15520,Three_meth_Othr_three_meth_P,Three_methods_Other_three_methods_Persons,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15520,G15521,Three_meth_Tot_three_meth_M,Three_methods_Total_three_methods_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15521,G15522,Three_meth_Tot_three_meth_F,Three_methods_Total_three_methods_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15522,G15523,Three_meth_Tot_three_meth_P,Three_methods_Total_three_methods_Persons,G59,G59,Persons,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15523,G15524,Worked_home_M,Worked_at_home_Males,G59,G59,Males,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over
15524,G15525,Worked_home_F,Worked_at_home_Females,G59,G59,Females,G59,Method of Travel to Work by Sex,Employed persons aged 15 years and over


In [10]:
# replace any words following the word "and" with a lowercase version
replace_ands = set()
for cats in df_meta_measures.Long.unique():
    try:
        word = re.search('(?<=_and_)\w+', cats).group(0).split('_')[0]
        replace_ands.add('_and_{}'.format(word))
    except:
        pass

for repl in replace_ands:
    df_meta_measures['Long'] = df_meta_measures['Long'].str.replace(repl,repl.lower())
    
# replace any words following the word "Occupation" with a lowercase version
replace_occs = set()
for cats in df_meta_measures.Long.unique():
    try:
        word = re.search('(?<=ccupation_)\w+', cats).group(0).split('_')[0]
        replace_occs.add('ccupation_{}'.format(word))
    except:
        pass

for repl in replace_occs:
    df_meta_measures['Long'] = df_meta_measures['Long'].str.replace(repl,repl.lower())

In [11]:
measure_cats = []
for category in df_meta_measures.Long.tolist():
    measure_cats.append(re.findall('[A-Z][^A-Z]*', category))

In [12]:
#export list to csv for framework to build reference table
with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(measure_cats)

### Begin working with new metaclasses file
This file has a kind of "OneHotEncoding" to assist infer which classes particular measures fall into, e.g. do they have detail for "Age", "Sex", "Place of Birth", etc.

In [2]:
df_meta = pd.read_csv('{}\Data\Metadata\Metadata_2016_refined.csv'.format(nb_path))

In [3]:
df_meta.head()

,Sequential,Short,Long,DataPack file,Profile table,Column heading description in profile,Table name,Table population,Age,Ancestry,...,Travel method type,Unemployed Status,Unpaid status,Volunteering status,Year of Arrival,Number of Classes,Number of Totals,Number of Classes Excl Total,Measures,Categories
0,G1,Tot_P_M,Total_Persons_Males,G01,G01a,Males,Selected Person Characteristics by Sex,Persons,0,0,...,0,0,0,0,0,1,0,1,Males|,Sex|
1,G2,Tot_P_F,Total_Persons_Females,G01,G01a,Females,Selected Person Characteristics by Sex,Persons,0,0,...,0,0,0,0,0,1,0,1,Females|,Sex|
2,G3,Tot_P_P,Total_Persons_Persons,G01,G01a,Persons,Selected Person Characteristics by Sex,Persons,0,0,...,0,0,0,0,0,1,1,0,Persons|,Total|
3,G4,Age_0_4_yr_M,Age_groups_0_4_years_Males,G01,G01a,Males,Selected Person Characteristics by Sex,Persons,1,0,...,0,0,0,0,0,2,0,2,Age_groups_0_4_years_|Males,Age|Sex
4,G5,Age_0_4_yr_F,Age_groups_0_4_years_Females,G01,G01a,Females,Selected Person Characteristics by Sex,Persons,1,0,...,0,0,0,0,0,2,0,2,Age_groups_0_4_years_|Females,Age|Sex


In [5]:
#Thoughts on algorithms to select line items
# For table in table_list
    # slice meta based on table
    # for category in filter_cats 
        # slice based on category >0
    # select rows with lowest value in "Number of Classes Excl Total" field
test_tbl = 'G57'
test_cats = ['Sex','Age'] # testing for rows that have a "Total" row first
meta_df_select = df_meta[df_meta['Profile table'].str[:3] == test_tbl].copy()
for cat in test_cats:
    meta_df_select = meta_df_select[meta_df_select[cat]>0]
min_fields = meta_df_select['Number of Classes Excl Total'].min()
meta_df_select = meta_df_select[meta_df_select['Number of Classes Excl Total'] == min_fields]
print(meta_df_select.shape)

(180, 73)


In [9]:
meta_df_select

,Sequential,Short,Long,DataPack file,Profile table,Column heading description in profile,Table name,Table population,Age,Ancestry,...,Travel method type,Unemployed Status,Unpaid status,Volunteering status,Year of Arrival,Number of Classes,Number of Totals,Number of Classes Excl Total,Measures,Categories
14836,G14837,M15_19_Tot,Males_15_19_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|15_19_years_|Total,Sex|Age|Total
14846,G14847,M20_24_Tot,Males_20_24_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|20_24_years_|Total,Sex|Age|Total
14856,G14857,M25_34_Tot,Males_25_34_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|25_34_years_|Total,Sex|Age|Total
14866,G14867,M35_44_Tot,Males_35_44_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|35_44_years_|Total,Sex|Age|Total
14876,G14877,M45_54_Tot,Males_45_54_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|45_54_years_|Total,Sex|Age|Total
14886,G14887,M55_64_Tot,Males_55_64_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|55_64_years_|Total,Sex|Age|Total
14896,G14897,M65_74_Tot,Males_65_74_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|65_74_years_|Total,Sex|Age|Total
14906,G14907,M75_84_Tot,Males_75_84_years_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|75_84_years_|Total,Sex|Age|Total
14916,G14917,M85_ov_Tot,Males_85_years_and_over_Total,G57A,G57,Total|MALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Males_|85_years_and_over_|Total,Sex|Age|Total
14936,G14937,F15_19_Tot,Females_15_19_years_Total,G57A,G57,Total|FEMALES,Occupation by Age by Sex,Employed persons aged 15 years and over,1,0,...,0,0,0,0,0,3,1,2,Females_|15_19_years_|Total,Sex|Age|Total


In [19]:
# Select the table file(s) to import
import_table_list = meta_df_select['DataPack file'].unique()
# Import the data table itself
df_data = load_census_csv(import_table_list, 'SA3')
# Select only columns included in the meta-sliced table above
df_data.set_index(df_data.columns[0], inplace=True)
refined_columns = meta_df_select.Short.tolist()
df_data = df_data[refined_columns]
df_data.shape

(358, 18)

In [50]:
# aggregate data by:
# transposing the dataframe
df_data_t = df_data.T.reset_index()
df_data_t.rename(columns={ df_data_t.columns[0]: 'Short' }, inplace = True)
# merging with the refined meta_df to give table name, "Measures" and "Categories" fields
meta_merge_ref = meta_df_select[['Short','Table name','Measures','Categories']]
df_data_t = df_data_t.merge(meta_merge_ref, on='Short')
df_data_t.head()

,Short,10102,10103,10104,10105,10106,10201,10202,10301,10302,...,89999,90101,90102,90103,90104,99797,99999,Table name,Measures,Categories
0,M15_19_Tot,826,269,757,478,456,1956,2004,565,631,...,3,4,3,3,22,0,0,Occupation by Age by Sex,Males_|15_19_years_|Total,Sex|Age|Total
1,M20_24_Tot,1252,422,883,714,618,3329,3289,1008,1017,...,18,27,7,14,18,0,0,Occupation by Age by Sex,Males_|20_24_years_|Total,Sex|Age|Total
2,M25_34_Tot,3219,873,1846,1422,1278,7090,6849,2181,2039,...,31,89,20,23,41,3,0,Occupation by Age by Sex,Males_|25_34_years_|Total,Sex|Age|Total
3,M35_44_Tot,3342,921,2288,1558,1751,8370,7196,2032,2141,...,20,95,31,16,93,0,3,Occupation by Age by Sex,Males_|35_44_years_|Total,Sex|Age|Total
4,M45_54_Tot,3793,1103,3138,1811,2005,8820,7566,2209,2504,...,15,115,28,17,111,0,0,Occupation by Age by Sex,Males_|45_54_years_|Total,Sex|Age|Total


In [57]:
#get positions in measures field for selected categories
position_list = []
for i, j in enumerate(df_data_t.Categories.iloc[0].split("|")):
    if j in test_cats:
        position_list.append(i)
print(position_list)

table_name = df_data_t['Table name'].iloc[0].replace(" ","_") + "|"

[0, 1]


'Occupation_by_Age_by_Sex:_Males__15_19_years_'

In [59]:
# from the "Categories" field, you should be able to split an individual entry by the "|" character
# to give the index of the measure you are interested in grouping by
# create a new column based on splitting the "Measure" field and selecting the value of this index/
    # or textjoining indices where there are multiple categories selected
    # also remove any instance of the word "total" and any multiple underscores
# Merge above with the table name to form "[Table_Name]|[groupby_value]" to have a good naming convention
# eg "Method_of_Travel_to_Work_by_Sex|Three_methods_Females"

#p = table_name + '_'.join([df_data_t.Measures.iloc[0].split("|")[i] for i in position_list])

def custom_name(table_namer, string_item, measure_positions): 
    return table_namer + '_'.join([string_item.split("|")[i] for i in measure_positions])

df_data_t['Test_name'] = df_data_t.apply(lambda x: custom_name(x['Table name'], x['Measures'], position_list), axis=1)
df_data_t.head()

,Short,10102,10103,10104,10105,10106,10201,10202,10301,10302,...,90101,90102,90103,90104,99797,99999,Table name,Measures,Categories,Test_name
0,M15_19_Tot,826,269,757,478,456,1956,2004,565,631,...,4,3,3,22,0,0,Occupation by Age by Sex,Males_|15_19_years_|Total,Sex|Age|Total,Occupation_by_Age_by_Sex:_Males__15_19_years_
1,M20_24_Tot,1252,422,883,714,618,3329,3289,1008,1017,...,27,7,14,18,0,0,Occupation by Age by Sex,Males_|20_24_years_|Total,Sex|Age|Total,Occupation_by_Age_by_Sex:_Males__20_24_years_
2,M25_34_Tot,3219,873,1846,1422,1278,7090,6849,2181,2039,...,89,20,23,41,3,0,Occupation by Age by Sex,Males_|25_34_years_|Total,Sex|Age|Total,Occupation_by_Age_by_Sex:_Males__25_34_years_
3,M35_44_Tot,3342,921,2288,1558,1751,8370,7196,2032,2141,...,95,31,16,93,0,3,Occupation by Age by Sex,Males_|35_44_years_|Total,Sex|Age|Total,Occupation_by_Age_by_Sex:_Males__35_44_years_
4,M45_54_Tot,3793,1103,3138,1811,2005,8820,7566,2209,2504,...,115,28,17,111,0,0,Occupation by Age by Sex,Males_|45_54_years_|Total,Sex|Age|Total,Occupation_by_Age_by_Sex:_Males__45_54_years_


In [ ]:
# then groupby this new column 
# then transpose again and either create the base data_df for future merges or merge with the already existing data_df
df_data_t = df_data_t.drop(['Short','Table name','Measures','Categories'], axis=1)
df_data_t = df_data_t.groupby(['Test_name']).sum()

In [65]:
df_data_t.head()

,10102,10103,10104,10105,10106,10201,10202,10301,10302,10303,...,80110,80111,89797,89999,90101,90102,90103,90104,99797,99999
Test_name,,,,,,,,,,,,,,,,,,,,,
Occupation_by_Age_by_Sex:_Females__15_19_years_,873,263,870,474,492,2423,2231,673,619,479,...,47,9,0,4,6,3,4,21,0,0
Occupation_by_Age_by_Sex:_Females__20_24_years_,1266,343,744,543,581,3014,3093,1025,799,600,...,168,9,0,19,21,3,3,13,0,0
Occupation_by_Age_by_Sex:_Females__25_34_years_,2895,753,1675,1371,1157,6326,5942,1835,1743,1429,...,618,39,0,27,65,22,11,57,0,0
Occupation_by_Age_by_Sex:_Females__35_44_years_,3236,864,2387,1427,1702,7578,6630,2015,2007,1718,...,338,49,0,22,64,15,16,104,0,4
Occupation_by_Age_by_Sex:_Females__45_54_years_,3686,1128,3421,1747,1992,8796,7380,2219,2433,2102,...,158,27,0,13,82,19,11,128,0,3


In [66]:
df_data = df_data_t.T
df_data.head()

Test_name,Occupation_by_Age_by_Sex:_Females__15_19_years_,Occupation_by_Age_by_Sex:_Females__20_24_years_,Occupation_by_Age_by_Sex:_Females__25_34_years_,Occupation_by_Age_by_Sex:_Females__35_44_years_,Occupation_by_Age_by_Sex:_Females__45_54_years_,Occupation_by_Age_by_Sex:_Females__55_64_years_,Occupation_by_Age_by_Sex:_Females__65_74_years_,Occupation_by_Age_by_Sex:_Females__75_84_years_,Occupation_by_Age_by_Sex:_Females__85_years_and_over_,Occupation_by_Age_by_Sex:_Males__15_19_years_,Occupation_by_Age_by_Sex:_Males__20_24_years_,Occupation_by_Age_by_Sex:_Males__25_34_years_,Occupation_by_Age_by_Sex:_Males__35_44_years_,Occupation_by_Age_by_Sex:_Males__45_54_years_,Occupation_by_Age_by_Sex:_Males__55_64_years_,Occupation_by_Age_by_Sex:_Males__65_74_years_,Occupation_by_Age_by_Sex:_Males__75_84_years_,Occupation_by_Age_by_Sex:_Males__85_years_and_over_
10102,873,1266,2895,3236,3686,2069,380,43,6,826,1252,3219,3342,3793,2421,633,86,11
10103,263,343,753,864,1128,792,258,39,0,269,422,873,921,1103,971,349,72,16
10104,870,744,1675,2387,3421,3174,646,61,7,757,883,1846,2288,3138,3210,927,145,29
10105,474,543,1371,1427,1747,1289,323,47,10,478,714,1422,1558,1811,1543,527,117,16
10106,492,581,1157,1702,1992,1371,413,72,12,456,618,1278,1751,2005,1560,635,154,27


In [35]:
# Consolidate into a single function
def refine_measure_name(table_namer, string_item, measure_positions):
    '''Simple function for generating measure names based on custom metadata information on ABS measures'''
    return table_namer + '|' + '_'.join([string_item.split("|")[i] for i in measure_positions])

def load_table_refined(table_ref, category_list, statistical_area_code='SA3'):
    '''
    Function for loading ABS census data tables, and refining/aggregating by a set of defined categories
    (e.g. age, sex, occupation, English proficiency, etc.) where available.
    
    INPUTS
    table_ref: STRING - the ABS Census Datapack table to draw information from (G01-G59)
    category_list: LIST of STRING objects - Cetegorical informatio to slice/aggregate information from (e.g. Age)
    statistical_area_code: STRING - the ABS statistical area level of detail required (SA1-SA3)
    '''
    df_meta = pd.read_csv('{}\Data\Metadata\Metadata_2016_refined.csv'.format(os.getcwd()))
    
    # slice meta based on table
    meta_df_select = df_meta[df_meta['Profile table'].str[:3] == table_ref].copy()
    
    # for category in filter_cats, slice based on category >0
    for cat in category_list:
        # First, check if there *are* any instances of the given category
        if meta_df_select[cat].sum() > 0:
            # If so, apply the filter
            meta_df_select = meta_df_select[meta_df_select[cat]>0]
        else:
            pass # If not, don't apply (otherwise you will end up with no selections)
        
    # select rows with lowest value in "Number of Classes Excl Total" field
    min_fields = meta_df_select['Number of Classes Excl Total'].min()
    meta_df_select = meta_df_select[meta_df_select['Number of Classes Excl Total'] == min_fields]
    
    # Select the table file(s) to import
    import_table_list = meta_df_select['DataPack file'].unique()
    
    # Import the SA data tables
    df_data = load_census_csv(import_table_list, statistical_area_code.upper())
    
    # Select only columns included in the meta-sliced table above
    df_data.set_index(df_data.columns[0], inplace=True)
    refined_columns = meta_df_select.Short.tolist()
    df_data = df_data[refined_columns]
    
    # aggregate data by:
    # transposing the dataframe
    df_data_t = df_data.T.reset_index()
    df_data_t.rename(columns={ df_data_t.columns[0]: 'Short' }, inplace = True)
    # merging with the refined meta_df to give table name, "Measures" and "Categories" fields
    meta_merge_ref = meta_df_select[['Short','Table name','Measures','Categories']]
    df_data_t = df_data_t.merge(meta_merge_ref, on='Short')
    
    position_list = []
    for i, j in enumerate(df_data_t.Categories.iloc[0].split("|")):
        if j in category_list:
            position_list.append(i)
    
    # from the "Categories" field, you should be able to split an individual entry by the "|" character
    # to give the index of the measure you are interested in grouping by
    # create a new column based on splitting the "Measure" field and selecting the value of this index/indices
    # Merge above with the table name to form "[Table_Name]|[groupby_value]" to have a good naming convention
    # eg "Method_of_Travel_to_Work_by_Sex|Three_methods_Females"
    df_data_t['Test_name'] = df_data_t.apply(lambda x: refine_measure_name(x['Table name'], x['Measures'], position_list), axis=1)
    
    # then groupby this new column 
    # then transpose again and either create the base data_df for future merges or merge with the already existing data_df
    df_data_t = df_data_t.drop(['Short','Table name','Measures','Categories'], axis=1)
    df_data_t = df_data_t.groupby(['Test_name']).sum()
    
    return df_data_t.T

In [36]:
# Test aggregation on import on category detail without "Total" fields
df_no_total = load_table_refined('G01', ['Education'])
df_no_total.head()

Test_name,Selected Person Characteristics by Sex|Did_not_go_to_school,Selected Person Characteristics by Sex|Year_10_or_equivalent,Selected Person Characteristics by Sex|Year_11_or_equivalent,Selected Person Characteristics by Sex|Year_12_or_equivalent,Selected Person Characteristics by Sex|Year_8_or_below,Selected Person Characteristics by Sex|Year_9_or_equivalent
10102,182,9514,2861,25229,1748,2482
10103,61,4192,1138,6856,761,1133
10104,138,17560,4970,22185,3337,5265
10105,141,8917,1913,10180,1714,2632
10106,112,7876,2071,11863,1553,2324


In [37]:
# Test import on table with no category detail matching
df_no_match = load_table_refined('G01', ['Occupation'])
df_no_match.head()

Test_name,Selected Person Characteristics by Sex|
10102,57331
10103,19366
10104,70638
10105,35559
10106,35942
10201,169053
10202,158683
10301,46594
10302,55336
10303,46612


In [ ]:
# Stitch it together to import multiple tables and multiple 

### Remaining Challenges
Ideally I would like to also have a mechanism to bring in sub-sets where available, and if not available still bring in the super-set (e.g. bring in "Religious Sect" detail where possible [Christianity] and for those religions without "Sect" breakdowns, still bring them in). Currently if you choose both "Religious Sect" and "Religion" as a category, you will get a double up of the "Total" lines of the "Religious Sect" subset, which could confuse some algorithms using the data, and would need to be manually picked out and removed.

It would also be nice to have a simple mechanism for showing which categories are available within which tables.